In [1]:
#!/usr/bin/env python3
import os

from datetime import timedelta
import dask
from dask.distributed import Client,default_client
from pathlib import Path
home = Path("/home/149/ab8992/tasman-tides")
gdata = Path("/g/data/nm03/ab8992")
import numpy as np
import xarray as xr
os.chdir(home)
import ttidelib as tt

def startdask():
    try:
    # Try to get the existing Dask client
        client = default_client()
        print(client)
    except ValueError:
        # If there's no existing client, create a new one
        client = Client(
            threads_per_worker = 2)
        print(client)
    return client

def save_data_for_filter(expt,zl,t0,tmpstorage,time_window = 300):

    client = startdask()
    # dask.config.set(scheduler="single-threaded")
    # tmpstorage = os.getenv('PBS_JOBFS')

    rawdata = tt.collect_data(
        expt,
        rawdata = ["u","v"],
        timerange=(t0 - time_window,t0 + time_window)).isel(zl = zl)

    ## Remove zl to make data properly 2D
    _rawdata = rawdata # Make a copy to modify the metadata to keep 
    _rawdata.u.attrs = {}
    _rawdata.v.attrs = {}
    _rawdata.zl.attrs = {}
    _rawdata.time.attrs = {}
    _rawdata.yb.attrs = {}
    _rawdata.xb.attrs = {}
    _rawdata = _rawdata.assign_coords({
        "time":_rawdata.time * 3600,
        "xb":_rawdata.xb * 1000,
        "yb":_rawdata.yb * 1000})
    _rawdata = _rawdata.drop_vars(["bathy","lat","lon"]) 
    if not os.path.exists(tmpstorage):
        os.makedirs(tmpstorage)
    print("Saving data to temporary storage")
    _rawdata.u.to_netcdf(tmpstorage + f"/u.nc",mode="w")
    _rawdata.v.to_netcdf(tmpstorage + f"/v.nc",mode="w")
    (_rawdata.v**2).rename("vv").to_netcdf(tmpstorage + f"/vv.nc",mode="w")
    (_rawdata.u**2).rename("uu").to_netcdf(tmpstorage + f"/uu.nc",mode="w")
    (_rawdata.u*_rawdata.v).rename("uv").to_netcdf(tmpstorage + f"/uv.nc",mode="w")
    print("done")
    client.close() ## Have to close dask client or it messes up the filtering package
    return

def lfilter(index):
    out = "/scratch/nm03/ab8992/lfilter_test40/test-parallel/" + str(index)
    tmpstorage = "/scratch/nm03/ab8992/lfilter_test"
    os.chdir("/scratch/nm03/ab8992/lfilter_test40/test-parallel/")
    time_window = 300
    filter_window = 2
    if os.path.exists(f"{out}.nc"):
        os.system(f"rm -r {out}.nc")
    # os.makedirs(out,exist_ok=True)
    data = xr.merge(
    [xr.open_dataset(tmpstorage + "/u.nc" ),
    xr.open_dataset(tmpstorage + "/v.nc" ),
    # xr.open_dataset(tmpstorage + "/uu.nc" ),
    # xr.open_dataset(tmpstorage + "/vv.nc" ),
    # xr.open_dataset(tmpstorage + "/uv.nc" )
    ]).isel(zl = index).load()
    f_high = filtering.LagrangeFilter(
        out, ## Save intermediate output to temporary storage
        data, 
        {"U": "u","V":"v"}, 
        {"lon": "xb", "lat": "yb", "time": "time","depth":"zl"},
        indices = {"depth": [index]},
        sample_variables=["U"], mesh="flat", highpass_frequency=2*np.pi/(15*3600),
        advection_dt=timedelta(minutes=5).total_seconds(),
        window_size=timedelta(hours=48).total_seconds(),
    )

    f_high(times=range(3600 * (150 - filter_window), 3600 * (150 + filter_window), 3600)) ## 
    return "hi"


# save_data_for_filter("full-40",slice(0,5),5000,"/scratch/nm03/ab8992/lfilter_test40")


In [3]:
lfilter(1)


--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: gadi-cpu-clx-0861
--------------------------------------------------------------------------
sh: None: command not found
INFO: Compiled ParcelsRandom ==> /jobfs/131655252.gadi-pbs/parcels-16078/libparcels_random_995e23b3-d9f2-4520-b9fd-e2516e470342.so
sh: None: command not found
INFO: Compiled SamplingParticlesample_kernel ==> /jobfs/131655252.gadi-pbs/parcels-16078/76d87ad634742a16132a5d096f7f7845_0.so
INFO:parcels.tools.loggers:Compiled SamplingParticlesample_kernel ==> /jobfs/131655252.gadi-pbs/parcels-16078/76d87ad634742a16132a5d096f7f7845_0.so
sh: None: command not found
INFO: Compiled SamplingParticleAdvectionRK4sample_kernel ==> /jobfs/131655252.gadi-pbs/parcels-16078/7af

'hi'

In [2]:
lfilter(1)

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: gadi-cpu-clx-0250
--------------------------------------------------------------------------
sh: None: command not found
INFO: Compiled ParcelsRandom ==> /jobfs/131537321.gadi-pbs/parcels-16078/libparcels_random_d35f39f0-ed8a-4d82-8e3c-11ae33fa8c95.so
sh: None: command not found
INFO: Compiled SamplingParticlesample_kernel ==> /jobfs/131537321.gadi-pbs/parcels-16078/518b3288c8fa7e9ccb178c0e30048caf_0.so
INFO:parcels.tools.loggers:Compiled SamplingParticlesample_kernel ==> /jobfs/131537321.gadi-pbs/parcels-16078/518b3288c8fa7e9ccb178c0e30048caf_0.so
sh: None: command not found
INFO: Compiled SamplingParticleAdvectionRK4sample_kernel ==> /jobfs/131537321.gadi-pbs/parcels-16078/e1f

'hi'

# Test first by reading in 20 layers and pre loading in chunks per z level

In [2]:
from dask import delayed, compute
import filtering

client = startdask()
# Create a list of delayed lfilter calls for different values of n
tasks = [delayed(lfilter)(n) for n in range(14)]

# Compute all tasks in parallel
results = compute(*tasks)

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: gadi-cpu-clx-0861
--------------------------------------------------------------------------
sh: None: command not found
INFO: Compiled ParcelsRandom ==> /jobfs/131655252.gadi-pbs/parcels-16078/libparcels_random_720e74bb-2a16-420f-b3a0-a8ad40cdabf7.so


<Client: 'tcp://127.0.0.1:40431' processes=14 threads=28, memory=112.00 GiB>


--------------------------------------------------------------------------

  Local host:   gadi-cpu-clx-0861
  Local device: mlx5_0
--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   gadi-cpu-clx-0861
  Local device: mlx5_0
--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   gadi-cpu-clx-0861
  Local device: mlx5_0
--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   gadi-cpu-clx-0861
  Local device: mlx5_0
--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   gadi-cpu-clx-0861
  Local device: mlx5_0
-----------------------------------

In [15]:
## Now load the data as dask array first

data = xr.merge(
    [xr.open_dataset(tmpstorage + "/u.nc", chunks={"zl":1,"time":-1,"yb":-1,"xb":-1}),
    xr.open_dataset(tmpstorage + "/v.nc", chunks={"zl":1,"time":-1,"yb":-1,"xb":-1})
    ]).persist()

## Test new lfilter function that can run in parallel